# Router: 질문을 적절한 곳으로 라우팅하기

이 노트북에서는 **질문의 유형에 따라 적절한 데이터 소스로 연결**하는 **Router** 기법을 배웁니다.

## Router란?

**Router**는 "교통 정리원"과 같습니다. 질문이 들어오면 어디로 보낼지 결정합니다.

### 일상 속 라우터 예시

```
┌────────────────────────────────────────────────────────────────────┐
│                   고객센터 전화 라우팅 예시                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   고객: "결제 문제가 있어요"                                       │
│          │                                                        │
│          ▼                                                        │
│   ┌─────────────┐                                                  │
│   │   Router    │  "결제" 키워드 감지!                             │
│   └──────┬──────┘                                                  │
│          │                                                        │
│          ▼                                                        │
│   💳 결제팀으로 연결                                               │
│                                                                    │
│   ---                                                             │
│                                                                    │
│   고객: "배송이 언제 오나요?"                                      │
│          │                                                        │
│          ▼                                                        │
│   ┌─────────────┐                                                  │
│   │   Router    │  "배송" 키워드 감지!                             │
│   └──────┬──────┘                                                  │
│          │                                                        │
│          ▼                                                        │
│   🚚 물류팀으로 연결                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### RAG에서의 Router

```
┌────────────────────────────────────────────────────────────────────┐
│                    RAG Router 예시                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   질문: "Python에서 리스트를 정렬하는 방법은?"                     │
│          │                                                        │
│          ▼                                                        │
│   ┌─────────────┐                                                  │
│   │   Router    │  "Python" 감지!                                 │
│   └──────┬──────┘                                                  │
│          │                                                        │
│          ▼                                                        │
│   🐍 Python 문서 벡터스토어에서 검색                               │
│                                                                    │
│   ---                                                             │
│                                                                    │
│   질문: "JavaScript에서 배열을 정렬하는 방법은?"                   │
│          │                                                        │
│          ▼                                                        │
│   ┌─────────────┐                                                  │
│   │   Router    │  "JavaScript" 감지!                             │
│   └──────┬──────┘                                                  │
│          │                                                        │
│          ▼                                                        │
│   🟨 JavaScript 문서 벡터스토어에서 검색                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama pydantic

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. Pydantic으로 라우팅 규칙 정의

## Pydantic이란?

**Pydantic**은 데이터의 형태를 정의하는 도구입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    Pydantic 쉬운 설명                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 딕셔너리:                                                   │
│   {"name": "홍길동", "age": "스물다섯"}  ← 나이가 문자열? 🤔      │
│                                                                    │
│   Pydantic 모델:                                                   │
│   class Person(BaseModel):                                         │
│       name: str      ← 이름은 문자열이어야 함                       │
│       age: int       ← 나이는 숫자여야 함                           │
│                                                                    │
│   Person(name="홍길동", age="스물다섯")  → ❌ 오류 발생!            │
│   Person(name="홍길동", age=25)         → ✅ 정상!                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

# 라우팅 규칙 정의
class RouteQuery(BaseModel):
    """
    질문을 어디로 라우팅할지 결정하는 모델
    
    datasource: 'python_docs' 또는 'js_docs' 중 하나만 선택 가능
    """
    datasource: Literal['python_docs', 'js_docs'] = Field(
        ...,  # 필수 값
        description='사용자 질문에 따라 가장 관련성 높은 데이터 소스를 선택합니다',
    )

print("✅ 라우팅 규칙 정의 완료")
print("   - python_docs: Python 관련 질문")
print("   - js_docs: JavaScript 관련 질문")

# 3. 구조화된 출력을 사용하는 Router 만들기

## with_structured_output이란?

LLM이 **정해진 형식으로만 응답**하게 만드는 기능입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│               with_structured_output 동작 방식                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 LLM 응답:                                                   │
│   "이 질문은 Python에 관한 것이므로 python_docs를 사용해야 합니다." │
│   → 텍스트에서 정보를 추출해야 함 😓                               │
│                                                                    │
│   구조화된 출력:                                                   │
│   RouteQuery(datasource='python_docs')                             │
│   → 바로 사용 가능! 😊                                             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# LLM 생성
llm = ChatOllama(model='llama3.2', temperature=0)

# 구조화된 출력을 반환하는 LLM
structured_llm = llm.with_structured_output(RouteQuery)

# 라우팅용 프롬프트
system = '''당신은 사용자 질문을 적절한 데이터 소스로 라우팅하는 전문가입니다. 
질문이 지목하는 프로그래밍 언어에 따라 해당 데이터 소스로 라우팅하세요.'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system),
    ('human', '{question}')
])

# 라우터 체인
router = prompt | structured_llm

print("✅ Router 체인 준비 완료")

# 4. Router 테스트

In [ ]:
# Python 관련 질문 테스트
question_python = '''이 코드가 안 돌아가는 이유를 설명해주세요: 
from langchain_core.prompts import ChatPromptTemplate 
prompt = ChatPromptTemplate.from_messages(['human', 'speak in {language}']) 
prompt.invoke('french')'''

result = router.invoke({'question': question_python})

print("=== Python 질문 라우팅 ===")
print(f"질문: {question_python[:50]}...")
print(f"라우팅 결과: {result}")
print(f"선택된 데이터소스: {result.datasource}")

In [ ]:
# JavaScript 관련 질문 테스트
question_js = '''JavaScript에서 배열의 map 메서드는 어떻게 사용하나요?
const numbers = [1, 2, 3];
// 어떻게 각 요소를 2배로 만들 수 있나요?'''

result = router.invoke({'question': question_js})

print("=== JavaScript 질문 라우팅 ===")
print(f"질문: {question_js[:50]}...")
print(f"라우팅 결과: {result}")
print(f"선택된 데이터소스: {result.datasource}")

# 5. 라우팅 결과에 따라 다른 체인 실행

## RunnableLambda란?

**RunnableLambda**는 일반 Python 함수를 LangChain 체인에서 사용할 수 있게 해줍니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                   RunnableLambda 역할                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 함수:                                                       │
│   def my_func(x):                                                  │
│       return x * 2                                                 │
│                                                                    │
│   RunnableLambda 적용:                                             │
│   chain = prompt | llm | RunnableLambda(my_func)                   │
│                                                                    │
│   → 일반 함수도 체인의 일부로 사용 가능!                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.runnables import RunnableLambda

def choose_route(result):
    """
    라우팅 결과에 따라 적절한 체인을 선택합니다.
    
    실제 프로젝트에서는 여기서 다른 Retriever나
    다른 RAG 체인을 반환하게 됩니다.
    """
    if 'python_docs' in result.datasource.lower():
        return '🐍 Python 문서 체인으로 이동합니다'
    else:
        return '🟨 JavaScript 문서 체인으로 이동합니다'

# 전체 라우팅 체인
full_chain = router | RunnableLambda(choose_route)

print("✅ 전체 라우팅 체인 준비 완료")

In [ ]:
# 전체 체인 테스트
print("=== 전체 라우팅 체인 테스트 ===")

print("\n--- Python 질문 ---")
result = full_chain.invoke({'question': question_python})
print(f"결과: {result}")

print("\n--- JavaScript 질문 ---")
result = full_chain.invoke({'question': question_js})
print(f"결과: {result}")

# 6. 다양한 질문으로 라우팅 테스트

In [ ]:
test_questions = [
    "Python의 리스트 컴프리헨션은 어떻게 작성하나요?",
    "JavaScript의 async/await 사용법을 알려주세요",
    "Flask에서 라우트를 정의하는 방법은?",
    "React에서 useState 훅은 어떻게 사용하나요?",
    "Django ORM에서 쿼리셋을 필터링하는 방법",
]

print("=== 다양한 질문 라우팅 테스트 ===")
for q in test_questions:
    result = full_chain.invoke({'question': q})
    print(f"\n질문: {q[:40]}...")
    print(f"결과: {result}")

---

## 정리: Router (라우터)

### Router의 핵심 구성요소

| 구성요소 | 역할 | 예시 |
|----------|------|------|
| **Pydantic 모델** | 라우팅 규칙 정의 | `Literal['python_docs', 'js_docs']` |
| **with_structured_output** | LLM이 구조화된 형식으로 응답 | `llm.with_structured_output(RouteQuery)` |
| **RunnableLambda** | 라우팅 결과 처리 | 조건에 따라 다른 체인 실행 |

### 핵심 코드

```python
# 1. 라우팅 규칙 정의
class RouteQuery(BaseModel):
    datasource: Literal['python_docs', 'js_docs']

# 2. 구조화된 출력 LLM
structured_llm = llm.with_structured_output(RouteQuery)

# 3. 라우터 체인
router = prompt | structured_llm

# 4. 조건부 실행
def choose_route(result):
    if result.datasource == 'python_docs':
        return python_chain
    else:
        return js_chain

full_chain = router | RunnableLambda(choose_route)
```

### 실제 활용 예시

| 상황 | 라우팅 대상 |
|------|------------|
| 프로그래밍 언어별 | Python 문서 / JS 문서 / Java 문서 |
| 주제별 | 기술 문서 / 마케팅 자료 / 법률 문서 |
| 난이도별 | 초급 자료 / 고급 자료 |
| 데이터 소스별 | 벡터DB / SQL DB / 웹 검색 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 변경
llm = ChatOllama(model='llama3.2', temperature=0)
```

## 다음 단계

**의미 기반으로 자동 라우팅**하는 **Semantic Router** 기법을 배웁니다. (21번 노트북)